In [1]:
import findspark
findspark.init("C:\Spark")
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
ss=SparkSession.builder.appName("Assignment").getOrCreate()

In [2]:
df=ss.read.csv("C:\Spark\walmart_stock.csv",header=True,inferSchema=True)

In [8]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [5]:
df.head(5)

[Row(Date=datetime.datetime(2012, 1, 3, 0, 0), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996),
 Row(Date=datetime.datetime(2012, 1, 4, 0, 0), Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475),
 Row(Date=datetime.datetime(2012, 1, 5, 0, 0), Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539),
 Row(Date=datetime.datetime(2012, 1, 6, 0, 0), Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922),
 Row(Date=datetime.datetime(2012, 1, 9, 0, 0), Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)]

In [7]:
df.select('open','high','low','volume').describe().show()

+-------+------------------+-----------------+-----------------+-----------------+
|summary|              open|             high|              low|           volume|
+-------+------------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|8222093.481717011|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|  4519780.8431556|
|    min|56.389998999999996|        57.060001|        56.299999|          2094900|
|    max|         90.800003|        90.970001|            89.25|         80898100|
+-------+------------------+-----------------+-----------------+-----------------+



In [15]:
df.filter('open>80').select('open').take(5)

[Row(open=80.059998),
 Row(open=80.440002),
 Row(open=80.550003),
 Row(open=81.16999799999999),
 Row(open=80.889999)]

In [34]:
df.select(max(df['open']),stddev(df['close'])).show()

+---------+------------------+
|max(open)|stddev_samp(close)|
+---------+------------------+
|90.800003| 6.756859163732991|
+---------+------------------+



In [39]:
df.filter((df['open']>80) & (df['close']<80)).show()

+-------------------+-----------------+---------+---------+---------+-------+---------+
|               Date|             Open|     High|      Low|    Close| Volume|Adj Close|
+-------------------+-----------------+---------+---------+---------+-------+---------+
|2013-12-09 00:00:00|        80.239998|    80.43|79.699997|79.949997|4491600|73.206501|
|2015-04-15 00:00:00|80.58000200000001|80.980003|79.650002|79.739998|6200400|75.292681|
+-------------------+-----------------+---------+---------+---------+-------+---------+



In [97]:
df1=df.withColumn('high*2',df['high']*2)
df1.select(['open','high','high*2']).head(5)

[Row(open=59.970001, high=61.060001, high*2=122.120002),
 Row(open=60.209998999999996, high=60.349998, high*2=120.699996),
 Row(open=59.349998, high=59.619999, high*2=119.239998),
 Row(open=59.419998, high=59.450001, high*2=118.900002),
 Row(open=59.029999, high=59.549999, high*2=119.099998)]

In [51]:
df.createOrReplaceTempView('walmart')

In [56]:
ss.sql('select open from walmart where open>90').show()

+---------+
|     open|
+---------+
|    90.32|
|90.800003|
+---------+



In [63]:
ss.sql('select sum(volume),date from walmart where open>90 group by date').show()

+-----------+-------------------+
|sum(volume)|               date|
+-----------+-------------------+
|    8522500|2015-01-09 00:00:00|
|    8215400|2015-01-13 00:00:00|
+-----------+-------------------+



In [68]:
ss.sql('select open from walmart where high<80 and low>75 order by open').head(5)

[Row(open=75.110001),
 Row(open=75.160004),
 Row(open=75.18),
 Row(open=75.209999),
 Row(open=75.260002)]

In [67]:
ss.sql('select count(date) from walmart where open>75 and close<80').show()

+-----------+
|count(date)|
+-----------+
|        346|
+-----------+



In [86]:
r=df.select(year(df['date']))
r.select(countDistinct('year(date)')).show()
r.createOrReplaceTempView('year')
l=ss.sql('select distinct(*) from year')

+--------------------------+
|count(DISTINCT year(date))|
+--------------------------+
|                         5|
+--------------------------+



In [93]:
l.orderBy('year(date)').show()

+----------+
|year(date)|
+----------+
|      2012|
|      2013|
|      2014|
|      2015|
|      2016|
+----------+

